# สร้าง data จาก spotify
    - จะทำการเก็บไฟล์ csv ของแต่ละหมวดเพลงไว้ในโฟลเดอร์ csv-file โดยมี all-genre ที่จะเก็บข้อมูลทุกหมวดหมู่ไว้
    - ในการเก็บข้อมูลไฟล์เพลง จะเก็บในรูป .wav โดยอยู่โฟลเดอร์ music_data โดยภายในจะแยกโฟลเดอร์ไว้แต่ละหมวดเพลง

In [1]:
!pip install python-dotenv
!pip install spotipy --upgrade


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import spotipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import time

#โหลดค่า client_id secret_id ที่ได้จาก spotify ผ่าน .env
load_dotenv()

client_credentials_manager = SpotifyClientCredentials(client_id=os.environ.get("SPOTIPY_CLIENT_ID"),
                                                      client_secret=os.environ.get("SPOTIPY_CLIENT_SECRET"))

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# ดึงข้อมูลจาก spotify
    ทำการดึงข้อมูลจาก spotify มาเก็บในรูปของ csv

In [3]:
#การเอาข้อมูลของเพลงโดยใช้ id ของเพลง
def get_genres_by_track_id(track_id):

    track_data = sp.track(track_id)

    artist_ids = []

    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])

    artists_data = sp.artists(artist_ids)

    genres = []

    for artist in artists_data["artists"]:
        genres += artist["genres"]
        break

    return genres

#เอาข้อมูลที่หมวดหมู่เพลงที่ spotify มีผ่านไฟล์ genres.json
def get_genres():
    # Open genres file
    try:
        with open('genres.json', 'r') as infile:
            valid_genres = json.load(infile)
    except FileNotFoundError:
        print("Couldn't find genres file!")
        sys.exit(1)
    
    return valid_genres

#สร้างไฟล์ csv ของเพลง
def create_csv(song_meta,genre,limit):
    index = 0
    song_meta_df = pd.DataFrame.from_dict(song_meta)
 
    # check the song feature
    features = sp.audio_features(song_meta['id'][0:50])
    for i in range(1,limit//50):
        features += sp.audio_features(song_meta['id'][50*i:50*i + 50])
        
    # change dictionary to dataframe
    features_df = pd.DataFrame.from_dict(features)

    # combine two dataframe
    final_df = song_meta_df.merge(features_df)

    final_df = final_df.drop(columns=['type','uri', 'track_href', 'time_signature'])

    final_df.to_csv(parent_dir_csv + genre + ".csv",index=False)

#ตรวจสอบว่ามีไฟล์ csv อยู่ในเครื่องแล้วหรือไม่
def check_file(genre):
    path  = parent_dir_csv + genre + ".csv"
    check_file = os.path.isfile(path)
    return check_file

In [4]:
genres_list = get_genres()

parent_dir_csv = "./csv-file/"

#id ของเพลง
ids=[]

#จำนวนเพลงที่ต้องการโหลดในแต่ละหมวด (ได้ไม่เกิน 1000 เพลงต่อหมวด)
limit_per_genres = 100

#list สำหรับเก็บว่าหมวดใดไม่สามารถสร้างได้ เนื่องจากบางหมวดไม่มีเพลงหรือไม่มี preview ให้
genres_not_create = []

In [5]:
if not os.path.exists(parent_dir_csv):
    os.mkdir(parent_dir_csv)


#ดึง id เพลงว่ามีเพลงใดเพิ่มไปแล้วบ้าง กรณีที่เกิด error ระหว่างการทำงาน
for genre in genres_list:
    if check_file(genre):
        ids += list(pd.read_csv(parent_dir_csv + genre + ".csv").id)
        continue
    break

ทำการดึงข้อมูลเพลงจาก spotify แล้ว save ในรูปแบบ csv ในแต่ละหมวดหมู่เพลง

In [6]:
for genre in genres_list:
    offset = 0
    song_meta={'id':[],'preview_url':[],'genre':[]}
    
    if check_file(genre):
        print("=======> have file {}.csv".format(genre))
        continue
        
    while list(song_meta.items())[2][1].count(genre) < limit_per_genres:
        
        print("{} : {}".format(genre,list(song_meta.items())[2][1].count(genre)))
        time.sleep(1)
        index = len(ids)
        try:
            results = sp.search(q="genre:"+genre,limit=50,offset=offset,type="track")
        except:
            break
            
        if (len(results['tracks']['items']) == 0):
            print("=======> result for search genre {} is 0 track".format(genre))
            break
        
        offset += 50
        # create a list of song ids
        for item in results['tracks']['items']:
            if item['id'] in ids:
                continue
            track = item['id']
            ids.append(track)
            
            if list(song_meta.items())[2][1].count(genre) == limit_per_genres:
                print("{} : {}".format(genre,list(song_meta.items())[2][1].count(genre)))
                break
            
            if item['preview_url'] == None:
                continue
                
            # song id
            song_meta['id'].append(item['id'])

            # song preview
            preview_url = item['preview_url']
            song_meta['preview_url'] += [preview_url]

            # song genres
            song_genre = genre
            song_meta["genre"] += [song_genre]
        
    if list(song_meta.items())[2][1].count(genre) == 0:
        print("=======> can't create file {}.csv".format(genre))
        genres_not_create.append(genre)
        continue
    
    print("=======> create csv {} {}".format(genre,list(song_meta.items())[2][1].count(genre)))
    create_csv(song_meta,genre,limit_per_genres)

=======> have file blues.csv
=======> have file jazz.csv
=======> have file opera.csv
=======> have file hip-hop.csv
=======> have file industrial.csv


In [7]:
#ทำการรวมไฟล์ csv ของแต่ละหมวดหมู่เพลงให้อยู่ในไฟล์เดียวกัน

#list all the files from the directory
file_list = os.listdir(parent_dir_csv)



if not check_file("all_genre"):
    df_all_genre = pd.DataFrame()
    #append all files together
    for file in file_list:
                df_temp = pd.read_csv(parent_dir_csv + file)
                df_all_genre = pd.concat([df_all_genre,df_temp], ignore_index=True)

    df_all_genre.to_csv(parent_dir_csv + "all_genre" + ".csv",index=False)

## ทำการสร้างไฟล์ wav ที่ได้จาก preview url ของแต่ละเพลง

In [8]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
import urllib.request
import numpy as np
from tqdm import tqdm

In [9]:
df_all_genre = pd.read_csv("./csv-file/all_genre.csv")

In [10]:
parent_dir_wav = "./music_data/"

if not os.path.exists(parent_dir_wav):
    os.mkdir(parent_dir_wav)

genres , counts = np.unique(df_all_genre['genre'],return_counts=True)
    
#ทำการกำหนด path และชื่อไฟล์ : parent_dir_wav/[genre]/[genre_name].wav
list_file_path = []
for genre,i in zip(genres , counts):
    genre_dir = os.path.join(parent_dir_wav, genre)
    for j in range(1,i+1):
        file_name = genre + "_" + str(j) + ".wav"
        list_file_path.append(genre_dir + "/" + file_name)

In [11]:
#ทำการ download preview music จาก url แล้วจัดเก็บในรูปไฟล์ wav
def download_url(args):
    url, fn = args[0], args[1]
    
    try:
        path = fn.rsplit("/",1)[0]
        file_name = fn.rsplit("/",1)[1]
    except Exception as e:
        print('Exception in download_url():', e)
        
    if not os.path.exists(path):
        os.mkdir(path)
        
    if os.path.isfile(path + "/" + file_name):
        return
    
    urllib.request.urlretrieve(url, path + "/" + file_name)

#ทำการ download แบบ parallel
def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)

***กรณีที่ไฟล์ไม่ครบให้ run ใหม่***

In [13]:
zip_music = zip(df_all_genre.iloc[0:].preview_url,list_file_path)
download_parallel(zip_music)